In [1]:
import pandas as pd
import numpy as np

In [2]:
L='fti tex tif sig hpq tjx sig fti sanm apc'.upper().split(' ')

In [9]:
first=np.array(pred['0'])
last=np.array(pred2['0'])

In [16]:
df=pd.read_csv('12_new_vwap.csv',header=0)


Merge the price data the same way as the quote and ask data, while for the size variable we sum over each 10 second period.

In [46]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

def VWAP_calc(ticker,capital):
    pred=pd.read_csv('./LSTM result December/'+ticker+'_20181217_pred.csv')
    pred2=pd.read_csv('./LSTM result December/'+ticker+'_20181231_pred.csv')
    first=np.array(pred['0'])
    last=np.array(pred2['0'])
    df_t=df[df['SYM_ROOT']==ticker]
    D=sorted(list(set(df_t['DATE'])))
    #print(vol_mean)
    df_d=df_t[df_t['DATE']==D[0]]
    T=[]
    s_start=[]
    p_start=[]
    i=-1
    for j in range(len(df_d['TIME_M'])):
        
        if len(T)>0 and (int(np.array(df_d['TIME_M'])[j][-15:-13])-int(T[-1][-2:]))>1:
            
                s_start.append(0)
                p_start.append(p_start[-1])
        if np.array(df_d['TIME_M'])[j][:-13] not in T:
            
            i+=1
            T.append(str(np.array(df_d['TIME_M'])[j][:-13]))
            p_start.append(np.array(df_d['PRICE'])[j])
            s_start.append(np.array(df_d['SIZE'])[j])
        else:
            s_start[i]+=np.array(df_d['SIZE'])[j]
    if '9:30' not in T:
        s_start=[0]+s_start
        p_start=[0]+p_start
    df_d=df_t[df_t['DATE']==D[-1]]
    T=[]
    s_end=[]
    p_end=[]
    i=-1
    for j in range(len(df_d['TIME_M'])):
        
        if len(T)>0 and (int(np.array(df_d['TIME_M'])[j][-15:-13])-int(T[-1][-2:]))>1:
           
                s_end.append(0)
                p_end.append(p_end[-1])
        if np.array(df_d['TIME_M'])[j][:-13] not in T:
            i+=1
            T.append(str(np.array(df_d['TIME_M'])[j][:-13]))
            p_end.append(np.array(df_d['PRICE'])[j])
            s_end.append(np.array(df_d['SIZE'])[j])
        else:
            s_end[i]+=np.array(df_d['SIZE'])[j]
    if '9:30' not in T:
        s_end=[0]+s_end
        p_end=[0]+p_end
    #print(T)
    Dict_=np.load('data.npy').item()
    p_d=np.array(Dict_[ticker.lower()]['Open'])[53]
    p_s=np.array(Dict_[ticker.lower()]['Open'])[62]
    cap=capital
    size=int(cap/p_d)
    hb,r,pb=execution(first,size,s_start,p_start)
    size=int(cap/p_d)
    hs,r,ps=execution(last,size,s_end,p_end)
    
    h,bb=benchmark(size,s_start,p_start)
    h,bs=benchmark(size,s_end,p_end)
    print('2018.12.17 (Buy)Decision Price:'+str(p_d)+' Executed VWAP:'+str(round(pb,3))+' Benchmark VWAP:'+str(round(bb,3)))
    print('2018.12.31 (Sell)Decision Price:'+str(p_s)+' Executed VWAP:'+str(round(ps,3))+' Benchmark VWAP:'+str(round(bs,3)))
    print('IS1:'+str(((pb-p_d)*size+(p_s-ps)*size)/float(capital)))
    return (size*ps-size*pb)/(size*pb),hb,hs,pb,ps,bb,bs,((bb-p_d)*size+(p_s-bs)*size)/float(capital),((pb-p_d)*size+(p_s-ps)*size)/float(capital)
   

In [47]:
shortfall=[]
benchmark_sf=[]
for i in set(L):

    print(i)
    record=VWAP_calc(i,L.count(i)*100000)
    IS=record[-1]*L.count(i)
    shortfall.append(IS)
    benchmark_sf.append(L.count(i)*record[-2])


TIF
2018.12.17 (Buy)Decision Price:81.4469 Executed VWAP:80.303 Benchmark VWAP:80.513
2018.12.31 (Sell)Decision Price:81.34 Executed VWAP:79.664 Benchmark VWAP:79.866
IS1:0.006517212541138244
APC
2018.12.17 (Buy)Decision Price:50.062 Executed VWAP:50.404 Benchmark VWAP:50.43
2018.12.31 (Sell)Decision Price:43.6963 Executed VWAP:43.745 Benchmark VWAP:43.764
IS1:0.005861912945845396
SIG
2018.12.17 (Buy)Decision Price:32.1531 Executed VWAP:31.727 Benchmark VWAP:31.774
2018.12.31 (Sell)Decision Price:30.991 Executed VWAP:31.266 Benchmark VWAP:31.217
IS1:-0.021807337097397534
TJX
2018.12.17 (Buy)Decision Price:45.7799 Executed VWAP:44.509 Benchmark VWAP:44.545
2018.12.31 (Sell)Decision Price:44.6543 Executed VWAP:44.279 Benchmark VWAP:44.238
IS1:-0.019556318955700803
SANM
2018.12.17 (Buy)Decision Price:24.28 Executed VWAP:24.083 Benchmark VWAP:24.191
2018.12.31 (Sell)Decision Price:23.92 Executed VWAP:23.853 Benchmark VWAP:23.878
IS1:-0.0053532148980949545
TEX
2018.12.17 (Buy)Decision Price

In [48]:
print('LSTM Shortfall:'+str(np.mean(shortfall)))
print('Benchmark Shortfall:'+str(np.mean(benchmark_sf)))
print('Period Theoretical Return:'+str(-0.037161334639139516))

LSTM Shortfall:-0.00012554924098891283
Benchmark Shortfall:0.001305008355129359
Period Theoretical Return:-0.037161334639139516


In [20]:
def execution(target_percentage,size,real_volume,real_price):
    i=0
    VWAP=0
    size_=size
    history=[]
    while size>0 and i<len(real_volume):
        goal=target_percentage[i]*size_
        if i==len(real_volume)-1:
            
            realized=min(size,real_volume[i])
        else:
            realized=min(goal,real_volume[i])
        realized=min(realized,size)
        VWAP+=realized*real_price[i]
        history.append(realized)
        
        i+=1
        size-=realized
    history=np.array(history)
    return history,size,VWAP/float(np.sum(history))
def benchmark(size,real_volume,real_price):
    history=np.array(real_volume)*size/float(sum(real_volume))
    VWAP=np.sum(history*np.array(real_price))
    return history,VWAP/float(np.sum(history))
